# Data Transformation

연봉 정보 데이터로,  
다양한 독립 변수를 활용해서 연봉을 예측할 수 있는 모델을 만들기 위한 데이터를 정제한다.

1. Data merge
2. Aggregation & Pivot table
4. Log
5. One Hot Encoding
6. Scaling
6. PCA




In [238]:
import pandas as pd
import numpy as np

# 1. Data merge (데이터 병합)


### pd.concat()

```python
pd.concat([df_a, df_b])  # 리스트 형태로 합칠 DataFrame 지정
pd.concat([df_a, df_b], axis=1)  # 옆으로 결합 가능 (있는 그대로 단순하게 결합)

```

기본적으로 데이터를 위아래로 이어붙이는 결합 방식  
동일한 컬럼은 한 줄로 이어져서 붙지만, 컬럼명이 다르면 각각의 컬럼으로 생성된다.  
특정 키 값을 기준으로 합치는 것이 아닌 단순하게 이어붙이는 방식  
인덱스는 원래 DataFrame에서 지정된 인덱스가 그대로 들어온다 -> 변경하고 싶다면 인덱스를 리셋해야 함


### merge()

```python
df_a.merge(df_b)  # 기본값 INNER JOIN
df_a.merge(df_b, how = 'outer')
df_a.merge(df_b, how = 'left')
df_a.merge(df_b, how = 'right')

df_a.merge(df_b, on='key')  # 기준 컬럼 지정 가능
df_a.merge(df_b, left_on = 'key', right_on = 'id')  # 공통 컬럼이 없는 경우
```
기본적으로 데이터를 옆으로 이어붙이는 결합 방식 (컬럼 기준)

### join()

```python
# merge()와 댜르게 공통된 컬럼에 suffix를 지정해줘야 한다.
df_a.join(df_b, lsuffix = '_a', rsuffix = '_b')

# 특정 컬럼(key)을 기준으로 이어붙이고 싶다면
# key값을 인덱스로 지정해야 한다.
df_a = df_a.set_index('key')
df_b = df_b.set_index('key')
df_a.join(df_b)

df_a.join(df_b])  # 기본값 INNER JOIN
df_a.join(df_b, how = 'outer')
df_a.join(df_b, how = 'left')
df_a.join(df_b, how = 'right')
```

기준이 되는 값을 인덱스에서 잡는다. (인덱스 기준)


In [239]:
salary_1 = pd.read_csv('/aiffel/data/salary_1.csv')
salary_2 = pd.read_csv('/aiffel/data/salary_2.csv')

In [240]:
salary_1.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0


In [241]:
salary_2.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,29,Female,1,Human Resources Coordinator,2,45000,China,Chinese,0
1,23,Male,0,Sales Associate,1,25000,USA,Asian,0
2,32,Female,2,Software Developer,7,90000,USA,African American,0
3,44,Male,1,Operations Manager,18,170000,USA,Hispanic,0
4,26,Female,1,Marketing Coordinator,2,40000,Canada,Asian,0


In [242]:
# salary_1과 salary_2 데이터 결합하기
salary_df = pd.concat([salary_1, salary_2])

In [243]:
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
2680,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0
2681,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0
2682,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0
2683,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0


In [244]:
salary_df.loc[0]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
0,29.0,Female,1,Human Resources Coordinator,2.0,45000,China,Chinese,0


In [245]:
# 중복된 인덱스를 처리하여 원래 DataFrame의 인덱스 형태로 변환하기

# drop = True: 생성된 인덱스 컬럼 제거
# inplace = True: salary_df에 덮어쓰기
salary_df.reset_index(drop = True, inplace = True)

In [246]:
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0
...,...,...,...,...,...,...,...,...,...
6679,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0
6680,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0
6681,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0
6682,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0


국가에 따라 연봉 차이가 클 수 있다.

국가별 물가 지수를 고려하면 연봉을 예측하는 모델을 만들 때, 유의미한 정보를 도출할 수 있을 것이라 예상된다.

In [247]:
# 소비가 물가 지수(cpi) 데이터 불러오기
cpi = pd.read_csv('/aiffel/data/cpi.csv')

In [248]:
cpi.head()

,Country,Reference,Last,Previous,Units,Frequency
0,United States,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
1,China,Sep-23,100,100.1,"Index CPPY=100, NSA",Monthly
2,Germany,Sep-23,117.9,117.5,"Index 2020=100, CDASA",Monthly
3,France,Sep-23,118.52,118.2,"Index 2015=100, SA",Monthly
4,United Kingdom,Sep-23,132,131.3,"Index 2015=100, NSA",Monthly


두 DataFrame에서 국가 이름이 어떻게 정의되어 있는지 확인해 보고, 불일치되는 값들은 변경한다.

In [249]:
salary_df['Country'].unique()

array(['UK', 'USA', 'Canada', 'China', 'Australia'], dtype=object)

In [250]:
cpi['Country'].unique()

array(['United States', 'China', 'Germany', 'France', 'United Kingdom',
       'Brazil', 'Italy', 'Russian Federation', 'Canada', 'Australia',
       'Spain', 'South Korea', 'Mexico', 'Indonesia', 'Netherlands'],
      dtype=object)

In [251]:
cpi['Country'].replace({'United States': 'USA', 'United Kingdom': 'UK'})

0                    USA
1                  China
2                Germany
3                 France
4                     UK
5                 Brazil
6                  Italy
7     Russian Federation
8                 Canada
9              Australia
10                 Spain
11           South Korea
12                Mexico
13             Indonesia
14           Netherlands
Name: Country, dtype: object

In [252]:
cpi['Country'] = cpi['Country'].replace({'United States': 'USA', 'United Kingdom': 'UK'})

In [253]:
# 파라미터 on을 써주는 습관을 들이는 것이 좋다.
# Country 컬럼 외에 공통된 컬럼이 없기 때문에 생략 가능
# left: salary_df가 메인 데이터이고, 여기에 국가별 cpi를 붙이는 것이기 때문
salary_df = salary_df.merge(cpi, on = 'Country', how = 'left')

In [254]:
salary_df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,Reference,Last,Previous,Units,Frequency
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,Sep-23,132,131.3,"Index 2015=100, NSA",Monthly
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,Sep-23,158.7,158.4,"Index 2002=100, SA",Monthly
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,Sep-23,307.48,306.27,"Index 1982-84=100, SA",Monthly


In [255]:
# 불필요한 컬럼 제거하기
salary_df.drop(['Reference','Previous','Units','Frequency'], axis = 1, inplace = True)

In [256]:
salary_df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,Last
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,132
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,158.7
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,307.48


In [257]:
# 추가한 컬럼명을 직관적으로 변경하기
# axis = 1 을 해주지 않으면 인덱스 값에서 찾게 된다.
salary_df = salary_df.rename({'Last': 'CPI'}, axis = 1)

In [258]:
salary_df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,132
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,158.7
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,307.48


① merge와 join의 특징 정리

    merge의 경우 공통된 컬럼이 있으면 suffix를 자동 생성한다.  
    동일한 이름의 컬럼을 구별하기 위해 사용된다.  
    join의 경우 공통된 컬럼이 있으면 suffix 수동 생성이 필요하다.  
    데이터 세트간의 컬럼 이름 충돌을 해결한다.  

② pandas에서는 merge 함수를 사용하여 SQL 스타일의 조인을 수행할 수 있다.

    Inner join: 공통키 반환 (*merge의 기본 설정)  
    Left join: 왼쪽, 모든 행, 오른쪽 일치 행 반환  
    Right join: 오른쪽, 모든 행, 왼쪽 일치 행 반환  
    Outer join: 두 행을 반환, 불일치 NaN 값으로 채워짐  

# 2. Agreegation & Pivot table

개요   
결측치&이상치(Missing Value & Outlier)  
집계 및 그룹화(Aggregation and Group by)  
피벗테이블(Pivot)



**① Overview**

데이터의 기본적인 특성을 파악하고, 데이터셋의 구조와 내용을 이해하는 데 도움을 준다. 

    head( ): 데이터프레임의 처음 몇 행(기본적으로 5행)을 표시한다. 데이터의 포맷, 변수의 형태 등을 빠르게 확인할 수 있다.  
    tail( ): 데이터프레임의 마지막 몇 행을 표시한다. 데이터의 전체적인 분포를 확인하는 데 도움이 된다.  
    describe( ): 수치형 열에 대한 기술통계를 제공한다. 평균, 표준편차, 최소값, 최대값 등을 확인할 수 있다.
    info( ): 데이터프레임의 기본 정보를 제공한다. 각 열의 데이터 타입, 누락된 값의 수, 메모리 사용량 등을 확인할 수 있다.  
    dtypes: 각 열의 데이터 타입을 보여준다. 데이터 형식을 빠르게 이해하는 데 유용하다.


**② Outlier**

이상치(Outlier)를 탐지하고 처리하는 데 사용되는 여러 방법과 함수들이 있다.

    통계적 방법: Z-점수와 IQR을 사용하여 이상치를 식별할 수 있다.  
    시각적 방법: 박스 플롯으로 이상치를 시각화하여 이상치를 식별할 수 있다.  
    데이터 필터링: 조건에 따라 이상치 필터링 및 제거등의 방법으로 이상치를 식별할 수 있다.


**③ Aggregation**

여러 데이터 포인트를 요약하고, 그룹화하여 새로운 통계 또는 정보를 추출하는 과정이다.

    groupby( ): 데이터를 특정 기준으로 그룹화하여 집계 연산에 적용할 수 있다.  
    집계 함수: sum( ), mean( ), median( ), min( ), max( ), count( ) 등으로 각 그룹의 요약 통계를 제공한다.  
    std( ) 함수 : 표준편차는 데이터 세트 내의 값들이 평균으로부터 얼마나 멀리 퍼져 있는지를 측정하는 값으로, 데이터의 분산 정도를 나타낸다.  
    agg( ) 함수: 다양한 집계 함수를 한 번에 적용하여 다양한 요약 통계를 얻을 수 있다.  


**④ Pivot Table**

데이터프레임을 재구성하여, 특정 열을 새로운 행과 열의 인덱스로 사용하고, 다른 열의 데이터로 새로운 표를 만드는 데 사용된다.

In [259]:
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6684 entries, 0 to 6683
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6680 non-null   float64
 1   Gender               6684 non-null   object 
 2   Education Level      6684 non-null   int64  
 3   Job Title            6684 non-null   object 
 4   Years of Experience  6684 non-null   float64
 5   Salary               6684 non-null   int64  
 6   Country              6684 non-null   object 
 7   Race                 6684 non-null   object 
 8   Senior               6684 non-null   int64  
 9   CPI                  6684 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 574.4+ KB


In [260]:
pd.to_numeric(salary_df['CPI'])

0       132.00
1       307.48
2       158.70
3       307.48
4       307.48
         ...  
6679    132.00
6680    135.30
6681    100.00
6682    100.00
6683    158.70
Name: CPI, Length: 6684, dtype: float64

CPI 컬럼에는 숫자 이외의 다른 문자가 섞여있던 것이 아니라, csv파일 특성때문에 object로 불러와졌음을 알 수 있다.

In [261]:
salary_df['CPI'] = pd.to_numeric(salary_df['CPI'])

In [262]:
salary_df.describe()

,Age,Education Level,Years of Experience,Salary,Senior,CPI
count,6680.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.00000
mean,33.611527,1.622382,8.084007,115307.175194,0.143477,167.12953
std,7.595506,0.880474,6.097824,52806.810881,0.350585,73.22657
min,21.000000,0.000000,-1.000000,350.000000,0.000000,100.00000
25%,28.000000,1.000000,3.000000,70000.000000,0.000000,132.00000
50%,32.000000,1.000000,7.000000,115000.000000,0.000000,135.30000
75%,38.000000,2.000000,12.000000,160000.000000,0.000000,158.70000
max,62.000000,3.000000,82.000000,250000.000000,1.000000,307.48000


## 2-1. 결측치 처리

In [263]:
salary_df.isna().sum()

Age                    4
Gender                 0
Education Level        0
Job Title              0
Years of Experience    0
Salary                 0
Country                0
Race                   0
Senior                 0
CPI                    0
dtype: int64

In [264]:
salary_df.isna().mean()

Age                    0.000598
Gender                 0.000000
Education Level        0.000000
Job Title              0.000000
Years of Experience    0.000000
Salary                 0.000000
Country                0.000000
Race                   0.000000
Senior                 0.000000
CPI                    0.000000
dtype: float64

In [265]:
salary_df[salary_df['Age'].isna()]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
548,NaN,Male,1,Data Analyst,3.0,130000,Canada,White,0,158.70
2135,NaN,Female,3,Project Engineer,16.0,190000,USA,African American,1,307.48
2749,NaN,Female,1,Software Engineer,1.0,50000,Australia,White,0,135.30
3826,NaN,Female,2,Marketing Coordinator,8.0,85000,UK,Asian,0,132.00


결측치가 크기 않기 때문에 mean이나 median으로 데이터를 채울 수 있지만,  
각 데이터의 경력에서 차이가 크기 때문에 데이터의 불균형을 초래할 수 있다. (노이즈)

-> 결측치 제거

In [266]:
salary_df = salary_df.dropna()

## 2-2. 이상치 처리

In [267]:
salary_df[salary_df['Years of Experience'] == -1]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
102,49.0,Female,2,HR Manager,-1.0,150000,Australia,White,1,135.3
256,35.0,Male,1,Project Coordinator,-1.0,95000,UK,Asian,1,132.0
359,34.0,Male,1,Operations Coordinator,-1.0,90000,China,Chinese,1,100.0


In [268]:
salary_df = salary_df[salary_df['Years of Experience'] != -1]

In [269]:
salary_df['Years of Experience'].sort_values()

4931     0.0
5104     0.0
5115     0.0
5119     0.0
5143     0.0
        ... 
2396    33.0
2391    33.0
2490    34.0
2415    34.0
564     82.0
Name: Years of Experience, Length: 6677, dtype: float64

In [270]:
salary_df[salary_df['Years of Experience'] == 82]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
564,25.0,Female,1,Data Analyst,82.0,110000,Australia,White,0,135.3


In [271]:
salary_df[(salary_df['Years of Experience'] > salary_df['Age'] - 18)]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
564,25.0,Female,1,Data Analyst,82.0,110000,Australia,White,0,135.3
2373,25.0,Male,3,Software Engineer Manager,12.0,169159,China,Chinese,0,100.0
2448,25.0,Male,3,Software Engineer Manager,12.0,169159,China,White,0,100.0


의심스러운 데이터는 제거한다.

In [272]:
salary_df = salary_df[~(salary_df['Years of Experience'] > salary_df['Age'] - 18)]

In [273]:
salary_df.describe()

,Age,Education Level,Years of Experience,Salary,Senior,CPI
count,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000
mean,33.612826,1.622116,8.076491,115294.402307,0.143093,167.164774
std,7.594369,0.880460,6.029750,52819.326323,0.350193,73.241730
min,21.000000,0.000000,0.000000,350.000000,0.000000,100.000000
25%,28.000000,1.000000,3.000000,70000.000000,0.000000,132.000000
50%,32.000000,1.000000,7.000000,115000.000000,0.000000,135.300000
75%,38.000000,2.000000,12.000000,160000.000000,0.000000,158.700000
max,62.000000,3.000000,34.000000,250000.000000,1.000000,307.480000


In [274]:
salary_df[salary_df['Years of Experience']  == 0]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
18,25.0,Female,1,Data Entry Clerk,0.0,35000,UK,Asian,0,132.00
49,25.0,Male,1,Help Desk Analyst,0.0,35000,USA,Asian,0,307.48
82,25.0,Male,1,Sales Representative,0.0,30000,Australia,Asian,0,135.30
2416,24.0,Male,2,Back end Developer,0.0,55538,USA,Asian,0,307.48
2430,22.0,Female,0,Back end Developer,0.0,51832,UK,White,0,132.00
...,...,...,...,...,...,...,...,...,...,...
6179,24.0,Female,0,Receptionist,0.0,25000,China,White,0,100.00
6193,24.0,Female,0,Receptionist,0.0,25000,Australia,Australian,0,135.30
6207,24.0,Female,0,Receptionist,0.0,25000,Australia,White,0,135.30
6221,24.0,Female,0,Receptionist,0.0,25000,UK,Welsh,0,132.00


나이를 봐도 Years of Experience가 0인 경우는 만으로 1년이 되지 않은 경우라고 생각하고 넘어갈 수 있다.

## 2-3. groupby()

In [275]:
# salary_df[salary_df['Gender'] == "Male"]['Salary'].mean()
# salary_df[salary_df['Gender'] == "Female"]['Salary'].mean()

salary_df.groupby('Gender').mean()

,Age,Education Level,Years of Experience,Salary,Senior,CPI
Gender,,,,,,
Female,32.622008,1.600066,7.417221,107873.854056,0.127992,167.485166
Male,34.425805,1.640207,8.617430,121383.057283,0.155483,166.901888


In [276]:
# groupby()로 특정 컬럼만 확인하기
salary_df.groupby('Gender')['Salary'].mean()

Gender
Female    107873.854056
Male      121383.057283
Name: Salary, dtype: float64

In [277]:
# 두 개 이상의 컬럼으로  groupby()하기
salary_df.groupby(['Gender','Country'])['Salary'].mean()

Gender  Country  
Female  Australia    107936.054010
        Canada       106884.711340
        China        111291.211506
        UK           108495.273026
        USA          104854.691558
Male    Australia    120896.764216
        Canada       123973.921516
        China        120135.522148
        UK           122244.048476
        USA          119683.120433
Name: Salary, dtype: float64

인덱스가 2개인 Pandas Series이다.  
Pandas DataFrame으로 변환하는 것도 가능하다.

In [278]:
# reset_index(): 인덱스가 컬럼 형태로 나온다.
salary_df.groupby(['Gender','Country'])['Salary'].mean().reset_index()

,Gender,Country,Salary
0,Female,Australia,107936.054010
1,Female,Canada,106884.711340
2,Female,China,111291.211506
3,Female,UK,108495.273026
4,Female,USA,104854.691558
5,Male,Australia,120896.764216
6,Male,Canada,123973.921516
7,Male,China,120135.522148
8,Male,UK,122244.048476
9,Male,USA,119683.120433


In [279]:
# 3개로 groupby()하는 것도 가능
salary_df.groupby(['Gender','Country', 'Race'])['Salary'].mean().reset_index()

,Gender,Country,Race,Salary
0,Female,Australia,Asian,107715.561905
1,Female,Australia,Australian,108245.014354
2,Female,Australia,White,107840.901042
3,Female,Canada,Asian,105812.939394
4,Female,Canada,Black,108602.208333
5,Female,Canada,White,106272.479167
6,Female,China,Chinese,106422.709184
7,Female,China,Korean,110948.144186
8,Female,China,White,117002.244444
9,Female,UK,Asian,106991.835714


In [280]:
# [참고] 두 개 이상의 연산으로 groupby()하기
salary_df.groupby('Gender')['Salary'].agg(['sum','mean'])

,sum,mean
Gender,,
Female,324484553,107873.854056
Male,444990288,121383.057283


## 2-4. pivot 생성하기

gropyby()에서랑 같은 값을 보여주지만,  

groupby()에서 Gender와 Country가 하나의 컬럼(인덱스)로 들어가있던 것을  
피벗테이블을 사용해 원하는 컬럼으로 행과 열을 정의해서 새로운 DataFrame을 짤 수 있다.

In [281]:
# 연산은 자동적으로 평균값이 구해진다.
pd.pivot_table(salary_df, index = 'Gender', columns = 'Country', values = 'Salary')

Country,Australia,Canada,China,UK,USA
Gender,,,,,
Female,107936.054010,106884.711340,111291.211506,108495.273026,104854.691558
Male,120896.764216,123973.921516,120135.522148,122244.048476,119683.120433


In [282]:
# aggfunc: 연산 지정 가능 (numpy 값으로도 가능)
pd.pivot_table(salary_df, index = 'Gender', columns = 'Country', values = 'Salary', aggfunc = 'mean')
# pd.pivot_table(salary_df, index = 'Gender', columns = 'Country', values = 'Salary', aggfunc = np.mean)

Country,Australia,Canada,China,UK,USA
Gender,,,,,
Female,107936.054010,106884.711340,111291.211506,108495.273026,104854.691558
Male,120896.764216,123973.921516,120135.522148,122244.048476,119683.120433


In [283]:
pd.pivot_table(salary_df, index = ['Gender','Race'], columns = 'Country', values = 'Salary', aggfunc = 'sum')

Country                   Australia      Canada       China          UK  \
Gender Race                                                               
Female African American         NaN         NaN         NaN         NaN   
       Asian             22620268.0  20950962.0         NaN  14978857.0   
       Australian        22623208.0         NaN         NaN         NaN   
       Black                    NaN  20851624.0         NaN         NaN   
       Chinese                  NaN         NaN  20858851.0         NaN   
       Hispanic                 NaN         NaN         NaN         NaN   
       Korean                   NaN         NaN  23853851.0         NaN   
       Mixed                    NaN         NaN         NaN  15673240.0   
       Welsh                    NaN         NaN         NaN  17882663.0   
       White             20705453.0  20404316.0  21060404.0  17430366.0   
Male   African American         NaN         NaN         NaN         NaN   
       Asian             31837948.0  31446774.0         NaN  24308468.0   
       Australian        29011770.0         NaN         NaN         NaN   
       Black                    NaN  30659174.0         NaN         NaN   
       Chinese                  NaN         NaN  28164994.0         NaN   
       Hispanic                 NaN         NaN         NaN         NaN   
       Korean                   NaN         NaN  29660738.0         NaN   
       Mixed                    NaN         NaN         NaN  23181267.0   
       Welsh                    NaN         NaN         NaN  19600032.0   
       White             26316849.0  29510780.0  31675232.0  21170436.0   

Country                         USA  
Gender Race                          
Female African American  15448789.0  
       Asian             17533654.0  
       Australian               NaN  
       Black                    NaN  
       Chinese                  NaN  
       Hispanic          14526773.0  
       Korean                   NaN  
       Mixed                    NaN  
       Welsh                    NaN  
       White             17081274.0  
Male   African American  23910271.0  
       Asian             20421501.0  
       Australian               NaN  
       Black                    NaN  
       Chinese                  NaN  
       Hispanic          21025292.0  
       Korean                   NaN  
       Mixed                    NaN  
       Welsh                    NaN  
       White             23088762.0

이렇게 구하는 것도 가능하다.

기본적으로 머신 러닝 모델을 만드는 목적이라면, 피벗테이블(pd.pivot_table) 보다는 groupby()를 사용하는 경우가 대부분이다.

데이터를 간결하게 원하는 형태로 보거나 Teblaue 등에 연결해 대시보드를 만들 때 피벗테이블이 필요한 경우도 있다.

-> 둘 다 유용한 함수라서 숙지하는 것이 좋다.

<br>

|  `pd.pivot_table` | `pd.pivot` |
|:--------:|:--------:|
| 연산이 가능 | 같은 형태인데 연산 없이 뿌려준다. |

`pd.melt()`을 사용해서 '피벗테이블 형태 -> DataFrame 형태'로 변환도 가능하다.

<br>


# 3. Log

### Log Transformation

데이터의 변환을 위해 로그 함수를 적용하는 과정  
데이터의 스케일을 조정하거나 정규성을 높이는 데 사용된다.


파이썬에서 로그를 취할 수 있는 간단한 방법은 numpy를 활용하는 것이다.

```python
np.log(x)    # 자연 로그
np.log10(x)  # 밑이 10인 로그
```

Linear Regression에서는 독립 변수와 종속 변수가 선형 관계에 있을수록 예측에 유리하기 때문에,  
이런 상황에서 log를 취하면 훨씬 좋은 예측이 나오게 된다.

```python
np.exp()     # 원래 값으로 되돌리기
```

# 4. One Hot Encoding

`get_dummies` : 주어진 범주형 열의 각 고유 범주를 대표하는 새로운 이진(0 또는 1) 열을 생성한다.

고윳값만큼의 새로운 변수들이 생겨난다.  

새로 생겨난 변수들 중 딱 하나의 변수는 제거할 수 있다.  
-> 연산의 부담을 줄이고, 큰 데이터에서 조금이라도 수를 줄이는 목적


선형 모델 등에서 디테일한 정보를 알고 싶을 때는 drop하지 않고 놔두는 것도 도움이 된다.

```python
pd.get_dummies()  # 원-핫 인코딩 적용
```

In [284]:
salary_df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,132.00
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,158.70
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,307.48


In [285]:
# One Hot Encoding
# drop_first = True: 하나의 값 제거하기 (알파벳순으로 첫 번째 값 제거)
pd.get_dummies(salary_df, columns = ['Gender'], drop_first = True)

,Age,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI,Gender_Male
0,32.0,1,Software Engineer,5.0,90000,UK,White,0,132.00,1
1,28.0,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48,0
2,45.0,3,Manager,15.0,150000,Canada,White,1,158.70,1
3,36.0,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48,0
4,52.0,2,Director,20.0,200000,USA,Asian,0,307.48,1
...,...,...,...,...,...,...,...,...,...,...
6679,49.0,3,Director of Marketing,20.0,200000,UK,Mixed,0,132.00,0
6680,32.0,0,Sales Associate,3.0,50000,Australia,Australian,0,135.30,1
6681,30.0,1,Financial Manager,4.0,55000,China,Chinese,0,100.00,0
6682,46.0,2,Marketing Manager,14.0,140000,China,Korean,0,100.00,1


In [286]:
pd.get_dummies(salary_df, columns = ['Gender','Country','Race'], drop_first = True)

,Age,Education Level,Job Title,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White
0,32.0,1,Software Engineer,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1
1,28.0,2,Data Analyst,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0
2,45.0,3,Manager,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1
3,36.0,1,Sales Associate,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,52.0,2,Director,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6679,49.0,3,Director of Marketing,20.0,200000,0,132.00,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6680,32.0,0,Sales Associate,3.0,50000,0,135.30,1,0,0,0,0,0,1,0,0,0,0,0,0,0
6681,30.0,1,Financial Manager,4.0,55000,0,100.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0
6682,46.0,2,Marketing Manager,14.0,140000,0,100.00,1,0,1,0,0,0,0,0,0,0,1,0,0,0


기본적으로 Pandas에서는 컬럼 수가 많아지면 일부 생략한다.  
생략된 컬럼도 확인하고 싶다면 옵션을 변경하여 컬럼을 늘릴 수 있다.

In [287]:
pd.set_option('display.max_columns', 50)

In [288]:
pd.get_dummies(salary_df, columns = ['Gender','Country','Race','Job Title'], drop_first = True)

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Job Title_Account Manager,Job Title_Accountant,Job Title_Administrative Assistant,Job Title_Advertising Coordinator,Job Title_Back end Developer,...,Job Title_Sales Operations Manager,Job Title_Sales Representative,Job Title_Scientist,Job Title_Social Media Man,Job Title_Social Media Manager,Job Title_Social Media Specialist,Job Title_Software Architect,Job Title_Software Developer,Job Title_Software Engineer,Job Title_Software Engineer Manager,Job Title_Software Manager,Job Title_Software Project Manager,Job Title_Strategy Consultant,Job Title_Supply Chain Analyst,Job Title_Supply Chain Manager,Job Title_Technical Recruiter,Job Title_Technical Support Specialist,Job Title_Technical Writer,Job Title_Training Specialist,Job Title_UX Designer,Job Title_UX Researcher,Job Title_VP of Finance,Job Title_VP of Operations,Job Title_Web Designer,Job Title_Web Developer
0,32.0,1,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,28.0,2,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,45.0,3,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36.0,1,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,52.0,2,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6679,49.0,3,20.0,200000,0,132.00,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6680,32.0,0,3.0,50000,0,135.30,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6681,30.0,1,4.0,55000,0,100.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6682,46.0,2,14.0,140000,0,100.00,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


카테고리 변수의 경우, 고윳값이 몇 개가 있냐에 따라 원-핫 인코딩의 결과가 다이나믹해질 수 있다.

따라서, 원-핫 인코딩을 할 때는 컬럼의 개수를 조절해 줄 필요가 있다.

In [289]:
salary_df['Gender'].unique()   # 고윳값 확인
salary_df['Gender'].nunique()  # 고윳값 개수 확인

2

In [290]:
salary_df[['Gender', 'Country', 'Race', 'Job Title']].nunique() 

Gender         2
Country        5
Race          10
Job Title    129
dtype: int64

In [291]:
salary_df['Job Title'].value_counts()
# salary_df['Job Title'].value_counts().tail(20)

Software Engineer                 808
Data Scientist                    515
Data Analyst                      389
Software Engineer Manager         374
Product Manager                   323
                                 ... 
Director of Sales                   1
UX Researcher                       1
Director of Product Management      1
Consultant                          1
Human Resources Director            1
Name: Job Title, Length: 129, dtype: int64

머신러닝 알고리즘을 적용해서 모델링을 할 때에는 케이스가 하나밖에 없는 것들은 큰 예측을 발휘하지 못한다.

In [292]:
salary_df['Job Title'].unique()

array(['Software Engineer', 'Data Analyst', 'Manager', 'Sales Associate',
       'Director', 'Marketing Analyst', 'Product Manager',
       'Sales Manager', 'Marketing Coordinator', 'Scientist',
       'Software Developer', 'HR Manager', 'Financial Analyst',
       'Project Manager', 'Customer Service Rep', 'Operations Manager',
       'Marketing Manager', 'Engineer', 'Data Entry Clerk',
       'Sales Director', 'Business Analyst', 'VP of Operations',
       'IT Support', 'Recruiter', 'Financial Manager',
       'Social Media Specialist', 'Software Manager', 'Developer',
       'Consultant', 'Product Designer', 'CEO', 'Accountant',
       'Data Scientist', 'Marketing Specialist', 'Technical Writer',
       'HR Generalist', 'Project Engineer', 'Customer Success Rep',
       'Sales Executive', 'UX Designer', 'Operations Director',
       'Network Engineer', 'Administrative Assistant',
       'Strategy Consultant', 'Copywriter', 'Account Manager',
       'Director of Marketing', 'Help Des

Job Title을 정제하여 Job으로 단순화시킨 데이터를 불러와서 사용해 보자.

In [293]:
job = pd.read_csv('/aiffel/data/job.csv')

In [294]:
job

,Job Title,Jobs
0,Software Engineer,Software
1,Data Analyst,Data Analyst
2,Manager,Manager
3,Sales Associate,Sales
4,Director,Director
...,...,...
123,Juniour HR Coordinator,HR
124,Digital Marketing Specialist,Marketing
125,Receptionist,Receptionist
126,Social Media Man,Social Media


In [295]:
# 'Job Title'이라는 key를 기준으로 데이터를 결합할 것이기 때문에 merge()를 사용
salary_df = salary_df.merge(job, on = 'Job Title', how = 'left')

In [296]:
salary_df

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Country,Race,Senior,CPI,Jobs
0,32.0,Male,1,Software Engineer,5.0,90000,UK,White,0,132.00,Software
1,28.0,Female,2,Data Analyst,3.0,65000,USA,Hispanic,0,307.48,Data Analyst
2,45.0,Male,3,Manager,15.0,150000,Canada,White,1,158.70,Manager
3,36.0,Female,1,Sales Associate,7.0,60000,USA,Hispanic,0,307.48,Sales
4,52.0,Male,2,Director,20.0,200000,USA,Asian,0,307.48,Director
...,...,...,...,...,...,...,...,...,...,...,...
6669,49.0,Female,3,Director of Marketing,20.0,200000,UK,Mixed,0,132.00,Marketing
6670,32.0,Male,0,Sales Associate,3.0,50000,Australia,Australian,0,135.30,Sales
6671,30.0,Female,1,Financial Manager,4.0,55000,China,Chinese,0,100.00,Financial
6672,46.0,Male,2,Marketing Manager,14.0,140000,China,Korean,0,100.00,Marketing


In [297]:
salary_df.drop('Job Title', axis = 1, inplace = True)

In [298]:
salary_df.head()

,Age,Gender,Education Level,Years of Experience,Salary,Country,Race,Senior,CPI,Jobs
0,32.0,Male,1,5.0,90000,UK,White,0,132.00,Software
1,28.0,Female,2,3.0,65000,USA,Hispanic,0,307.48,Data Analyst
2,45.0,Male,3,15.0,150000,Canada,White,1,158.70,Manager
3,36.0,Female,1,7.0,60000,USA,Hispanic,0,307.48,Sales
4,52.0,Male,2,20.0,200000,USA,Asian,0,307.48,Director


In [299]:
salary_df['Jobs'].nunique()

60

129개에서 60개로 줄어든 것을 확인할 수 있다.

원-핫 인코딩의 결과를 그대로 사용하는 것보다는 훨씬 더 적은 컬럼수를 유지할 수 있다.

In [300]:
salary_df['Jobs'].value_counts()

Software                        2156
Marketing                        999
Data Scientist                   572
Sales                            453
HR                               420
Data Analyst                     389
Product Manager                  325
Project Engineer                 316
Financial                        200
Research                         195
Operations                       147
Web Developer                    129
Product Designer                  80
Receptionist                      57
Project Manager                   36
Business Analyst                  23
Graphic Designer                  23
Social Media                      18
Customer Service                  11
Business Development               7
UX                                 6
Accountant                         6
Business Development Manager       5
Account                            5
Delivery Driver                    5
Project Coordinator                4
Scientist                          4
R

케이스가 많은 컬럼과 케이스가 많지 않는 경우를 others 이렇게 딱 2개로 바꾸게 되면,  
정보 손실이 크지 않으면서 원-핫 인코딩을 통해 컬럼수도 크게 증가하지 않는 효과를 볼 수 있다.

In [301]:
salary_df.head()

,Age,Gender,Education Level,Years of Experience,Salary,Country,Race,Senior,CPI,Jobs
0,32.0,Male,1,5.0,90000,UK,White,0,132.00,Software
1,28.0,Female,2,3.0,65000,USA,Hispanic,0,307.48,Data Analyst
2,45.0,Male,3,15.0,150000,Canada,White,1,158.70,Manager
3,36.0,Female,1,7.0,60000,USA,Hispanic,0,307.48,Sales
4,52.0,Male,2,20.0,200000,USA,Asian,0,307.48,Director


In [302]:
pd.get_dummies(salary_df, columns = ['Gender','Country','Race','Jobs'], drop_first = True)

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,32.0,1,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,28.0,2,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,45.0,3,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36.0,1,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,52.0,2,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6669,49.0,3,20.0,200000,0,132.00,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6670,32.0,0,3.0,50000,0,135.30,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6671,30.0,1,4.0,55000,0,100.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6672,46.0,2,14.0,140000,0,100.00,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


스케일링을 통해 148개의 컬럼이 79개로 줄어든 것을 확인할 수 있다. (이것도 많기는 함)

In [303]:
salary_df = pd.get_dummies(salary_df, columns = ['Gender','Country','Race','Jobs'], drop_first = True)

# 5. Scailing

**① Scaling**

    데이터의 범위를 조정하는 과정  
    MinMaxScaler, StandardScaler 등을 사용해 특성의 스케일을 조정한다.  
    거리기반 모델을 쓸 때는 반드시 스케일링 써서 각각 변수의 특성에 따라 동일한 스케일을 지닐수 있게 맞춰주는것이 중요하다.  

**② 주요 Scaling 방법**

- Standardization (Standard Scaling): 각 데이터 포인트에서 평균을 빼고, 그 결과를 표준편차로 나누어 준다. 결과적으로 데이터는 평균이 0이고 표준편차가 1인 분포를 갖게 된다.  
    $z = \frac{x_i - \mathrm{mean}(x)}{\mathrm{sd}(x)}$


- Robust Scaling : 이 방법은 중앙값과 사분위 범위를 사용하여 데이터를 스케일링한다. 표준화와 유사하지만, 이상치의 영향을 덜 받는다.  
    $z = \frac{x_i - Q_2(x)}{Q_3(x) - Q_1(x)}$


- MinMaxScaler : 데이터를 0과 1 사이의 범위로 조정한다. 주로 최소값과 최대값을 사용하여 계산한다. 주로 MinMaxScaler라는 클래스를 통해 구현된다.  
    $z = \frac{x_i - \min(x)}{\max(x) - \min(x)}$


**③ 각 Scaling 방법의 적합한 상황과 특징**

1. Standardization : 아웃라이어가 없는 경우나, 변수의 분포가 정규분포를 따를 때 적합하다.
- 리니어 리그레이션(연속적인 값 예측), 로지스틱 회귀(분류 문제에 적합한 확률 결과 제공), 서포트 벡터 머신(이진 또는 다중 클래스 분류, 회귀)에 사용된다.
- 주의사항 : 아웃라이어에 민감하므로, 아웃라이어가 존재하지 않을 때 더 효과적이다.
2. Robust Scaling : 아웃라이어가 많은 데이터에 적합하다.
- 특징 : 정규분포를 따르지 않는 데이터에 적용하기 좋은 방법이다.
3. MinMaxScaler : 데이터의 범위를 0에서 1로 제한하고자 할 때 적합하다.
- 주로 딥러닝 모델에서 사용된다.
- 특징 : 스케일링 범위를 제한하고자 할 때 유용하다.

### Age에 Standard Scaling 적용

In [304]:
(salary_df['Age'] - salary_df['Age'].mean()) / salary_df['Age'].std()

0      -0.212371
1      -0.739077
2       1.499423
3       0.314335
4       2.421159
          ...   
6669    2.026129
6670   -0.212371
6671   -0.475724
6672    1.631100
6673   -1.002430
Name: Age, Length: 6674, dtype: float64

### Age에 Robust Scaling 적용

In [305]:
# 사분위수 Q값 구하기
salary_df['Age'].quantile(0.75)

38.0

In [307]:
(salary_df['Age'] - salary_df['Age'].quantile(0.5)) / (salary_df['Age'].quantile(0.75) - salary_df['Age'].quantile(0.25))

0       0.0
1      -0.4
2       1.3
3       0.4
4       2.0
       ... 
6669    1.7
6670    0.0
6671   -0.2
6672    1.4
6673   -0.6
Name: Age, Length: 6674, dtype: float64

### Age에 MinMaxScaler 적용

In [308]:
(salary_df['Age'] - salary_df['Age'].min()) / (salary_df['Age'].max() - salary_df['Age'].min())

0       0.268293
1       0.170732
2       0.585366
3       0.365854
4       0.756098
          ...   
6669    0.682927
6670    0.268293
6671    0.219512
6672    0.609756
6673    0.121951
Name: Age, Length: 6674, dtype: float64

In [309]:
salary_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,32.0,1,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,28.0,2,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,45.0,3,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36.0,1,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,52.0,2,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 스케일러 함수로 Scaling 적용하기

In [310]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [311]:
ss = StandardScaler()
rs = RobustScaler()
mm = MinMaxScaler()

In [312]:
# 학습시키기 - 연산에 필요한 정보들을 습득 (평균, 표준편차 등)
ss.fit(salary_df)

StandardScaler()

In [316]:
# 연산하기
# numpy array 형태로 출력되기 때문에 DataFrame 형태로 변환해 주기
# numpy array 형태에는 컬럼 이름이 지정되어 있지 않기 때문에 지정해 주기
pd.DataFrame(ss.transform(salary_df), columns = salary_df.columns)

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,-0.212387,-0.706633,-0.510257,-0.478921,-0.408641,-0.480155,0.905822,-0.496767,-0.500281,2.004506,-0.504725,-0.560853,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,1.555332,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,1.447601,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
1,-0.739133,0.429222,-0.841970,-0.952268,-0.408641,1.915926,-1.103970,-0.496767,-0.500281,-0.498876,1.981278,-0.560853,-0.269528,-0.264051,-0.265993,4.441476,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
2,1.499536,1.565077,1.148311,0.657112,2.447137,-0.115582,0.905822,2.013016,-0.500281,-0.498876,-0.504725,-0.560853,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,1.555332,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
3,0.314358,-0.706633,-0.178543,-1.046938,-0.408641,1.915926,-1.103970,-0.496767,-0.500281,-0.498876,1.981278,-0.560853,-0.269528,-0.264051,-0.265993,4.441476,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,3.705792,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
4,2.421340,0.429222,1.977595,1.603806,-0.408641,1.915926,0.905822,-0.496767,-0.500281,-0.498876,1.981278,1.782999,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6669,2.026281,1.565077,1.977595,1.603806,-0.408641,-0.480155,-1.103970,-0.496767,-0.500281,2.004506,-0.504725,-0.560853,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,4.356838,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314

In [317]:
ss_df = pd.DataFrame(ss.transform(salary_df), columns = salary_df.columns)

In [318]:
rs.fit(salary_df)

RobustScaler()

In [319]:
rs_df = pd.DataFrame(rs.transform(salary_df), columns = salary_df.columns)

In [320]:
mm.fit(salary_df)

MinMaxScaler()

In [321]:
mm_df = pd.DataFrame(mm.transform(salary_df), columns = salary_df.columns)

In [322]:
ss_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,-0.212387,-0.706633,-0.510257,-0.478921,-0.408641,-0.480155,0.905822,-0.496767,-0.500281,2.004506,-0.504725,-0.560853,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,1.555332,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,1.447601,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
1,-0.739133,0.429222,-0.841970,-0.952268,-0.408641,1.915926,-1.103970,-0.496767,-0.500281,-0.498876,1.981278,-0.560853,-0.269528,-0.264051,-0.265993,4.441476,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
2,1.499536,1.565077,1.148311,0.657112,2.447137,-0.115582,0.905822,2.013016,-0.500281,-0.498876,-0.504725,-0.560853,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,1.555332,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
3,0.314358,-0.706633,-0.178543,-1.046938,-0.408641,1.915926,-1.103970,-0.496767,-0.500281,-0.498876,1.981278,-0.560853,-0.269528,-0.264051,-0.265993,4.441476,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,3.705792,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391
4,2.421340,0.429222,1.977595,1.603806,-0.408641,1.915926,0.905822,-0.496767,-0.500281,-0.498876,1.981278,1.782999,-0.269528,-0.264051,-0.265993,-0.225150,-0.271124,-0.229524,-0.229162,-0.642950,-0.029997,-0.017314,-0.012242,-0.058806,-0.012242,...,-0.012242,-0.012242,-0.150073,-0.012242,-0.110146,-0.22625,-0.024489,-0.222938,-0.073643,-0.012242,-0.092813,-0.024489,-0.173486,-0.269848,-0.024489,-0.052003,-0.690798,-0.012242,-0.012242,-0.021206,-0.012242,-0.017314,-0.029997,-0.012242,-0.140391


원-핫 인코딩으로 처리된 값들이 0과 1로만 이루진 컬럼인데도 각 컬럼마다 다른 값들로 나온 것을 확인할 수 있다.  
-> 0과 1이 어떤 비중을 차지하고 있냐에 따라서 평균과 표준편차가 달라지기 때문

In [323]:
rs_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,0.0,0.0,-0.222222,-0.277778,0.0,-0.123596,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.4,1.0,-0.444444,-0.555556,0.0,6.448689,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.3,2.0,0.888889,0.388889,1.0,0.876404,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.4,0.0,0.000000,-0.611111,0.0,6.448689,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,1.0,1.444444,0.944444,0.0,6.448689,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [324]:
mm_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,0.268293,0.333333,0.147059,0.359103,0.0,0.154232,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.170732,0.666667,0.088235,0.258963,0.0,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.585366,1.000000,0.441176,0.599439,1.0,0.282919,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.365854,0.333333,0.205882,0.238935,0.0,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.756098,0.666667,0.588235,0.799720,0.0,1.000000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [325]:
salary_df.head()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,32.0,1,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,28.0,2,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,45.0,3,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36.0,1,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,52.0,2,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


MinMaxScaling에서는 사실상 원-핫 인코딩 했던 결과 그대로 나왔다고 볼 수 있다.

In [326]:
ss_df.describe()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
count,6.674000e+03,6.674000e+03,6674.000000,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,...,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6674.000000,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03,6.674000e+03
mean,3.406858e-17,1.277572e-16,0.000000,1.022057e-16,1.022057e-16,-2.571113e-16,-1.192400e-16,-7.186340e-17,-3.619786e-17,-6.427782e-17,-2.235750e-17,-4.152108e-17,3.460090e-17,3.832715e-17,-6.387858e-18,4.870742e-17,5.323215e-18,-3.327009e-17,3.193929e-17,4.471501e-17,-4.258572e-18,2.129286e-18,-1.064643e-18,-2.981000e-17,-1.064643e-18,...,-2.129286e-18,-2.129286e-18,-2.555143e-17,-1.064643e-18,-3.406858e-17,2.555143e-17,-8.517144e-18,6.813715e-17,3.406858e-17,-1.064643e-18,-1.703429e-17,-1.064643e-17,0.000000,3.406858e-17,-1.064643e-17,3.406858e-17,-3.406858e-17,-2.129286e-18,-1.064643e-18,4.258572e-18,-2.129286e-18,-2.129286e-18,-4.258572e-18,-2.129286e-18,-4.258572e-17
std,1.000075e+00,1.000075e+00,1.000075,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,...,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00,1.000075e+00
min,-1.660937e+00,-1.842488e+00,-1.339541,-2.176344e+00,-4.086407e-01,-9.170975e-01,-1.103970e+00,-4.967671e-01,-5.002809e-01,-4.988760e-01,-5.047248e-01,-5.608528e-01,-2.695282e-01,-2.640506e-01,-2.659933e-01,-2.251504e-01,-2.711238e-01,-2.295243e-01,-2.291623e-01,-6.429496e-01,-2.999700e-02,-1.731358e-02,-1.224164e-02,-5.880584e-02,-1.224164e-02,...,-1.224164e-02,-1.224164e-02,-1.500728e-01,-1.224164e-02,-1.101465e-01,-2.262502e-01,-2.448878e-02,-2.229376e-01,-7.364320e-02,-1.224164e-02,-9.281258e-02,-2.448878e-02,-0.173486,-2.698479e-01,-2.448878e-02,-5.200314e-02,-6.907983e-01,-1.224164e-02,-1.224164e-02,-2.120631e-02,-1.224164e-02,-1.731358e-02,-2.999700e-02,-1.224164e-02,-1.403912e-01
25%,-7.391327e-01,-7.066332e-01,-0.841970,-8.575989e-01,-4.086407e-01,-4.801554e-01,-1.103970e+00,-4.967671e-01,-5.002809e-01,-4.988760e-01,-5.047248e-01,-5.608528e-01,-2.695282e-01,-2.640506e-01,-2.659933e-01,-2.251504e-01,-2.711238e-01,-2.295243e-01,-2.291623e-01,-6.429496e-01,-2.999700e-02,-1.731358e-02,-1.224164e-02,-5.880584e-02,-1.224164e-02,...,-1.224164e-02,-1.224164e-02,-1.500728e-01,-1.224164e-02,-1.101465e-01,-2.262502e-01,-2.448878e-02,-2.229376e-01,-7.364320e-02,-1.22

In [327]:
round(3.406858e-17, 10)

0.0

In [ ]:
1.000075e+00

StandardScaling의 결과

- mean 값은 모든 컬럼에서 거의 0에 근접하게 나온다. 즉, mean 값은 모두 0이라고 볼 수 있다.

- std 값은 모든 컬럼에서 거의 1에 근접하게 나온다.

평균 0, 표준편차 1, min/max 값은 컬럼의 특성(데이터의 분포)에 따라 조금씩 다르게 나온다.


In [329]:
rs_df.describe()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
count,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.00000,6674.000000,6674.000000,6674.000000,...,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.00000,6674.000000,6674.000000,6674.000000
mean,0.161283,0.622116,0.119610,0.003271,0.143093,1.193437,-0.450704,0.197932,0.200180,0.199281,0.203027,0.239287,0.067726,0.065178,0.066077,0.048247,0.068475,0.050045,0.049895,0.292478,0.000899,0.00030,0.000150,0.003446,0.000150,...,0.000150,0.000150,0.022026,0.000150,0.011987,0.048696,0.000599,0.047348,0.005394,0.000150,0.008541,0.000599,0.029218,0.067875,0.000599,0.002697,0.323045,0.000150,0.000150,0.000450,0.000150,0.00030,0.000899,0.000150,0.019329
std,0.759437,0.880460,0.669972,0.586881,0.350193,2.743136,0.497601,0.398471,0.400165,0.399490,0.402282,0.426680,0.251293,0.246859,0.248436,0.214304,0.252578,0.218054,0.217745,0.454935,0.029972,0.01731,0.012241,0.058608,0.012241,...,0.012241,0.012241,0.146778,0.012241,0.108834,0.215249,0.024476,0.212398,0.073251,0.012241,0.092027,0.024476,0.168429,0.251551,0.024476,0.051867,0.467675,0.012241,0.012241,0.021198,0.012241,0.01731,0.029972,0.012241,0.137688
min,-1.100000,-1.000000,-0.777778,-1.273889,0.000000,-1.322097,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,-0.400000,0.000000,-0.444444,-0.500000,0.000000,-0.123596,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.600000,1.000000,0.555556,0.500000,0.000000,0.876404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000

RobertScaling의 결과

- 평균이 0에 꽤 근접하나 완전히 0으로 정리되지는 않는다.
- 표준편차 또한 1에 꽤 근접하나 완전히 1로 정리되지는 않는다.

min/max 값은 컬럼의 특성(데이터의 분포)에 따라 조금씩 다르게 나온다.

In [330]:
mm_df.describe()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
count,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.00000,6674.000000,6674.000000,6674.000000,...,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.00000,6674.000000,6674.000000,6674.000000
mean,0.307630,0.540705,0.237544,0.460422,0.143093,0.323717,0.549296,0.197932,0.200180,0.199281,0.203027,0.239287,0.067726,0.065178,0.066077,0.048247,0.068475,0.050045,0.049895,0.292478,0.000899,0.00030,0.000150,0.003446,0.000150,...,0.000150,0.000150,0.022026,0.000150,0.011987,0.048696,0.000599,0.047348,0.005394,0.000150,0.008541,0.000599,0.029218,0.067875,0.000599,0.002697,0.323045,0.000150,0.000150,0.000450,0.000150,0.00030,0.000899,0.000150,0.019329
std,0.185229,0.293487,0.177346,0.211574,0.350193,0.353006,0.497601,0.398471,0.400165,0.399490,0.402282,0.426680,0.251293,0.246859,0.248436,0.214304,0.252578,0.218054,0.217745,0.454935,0.029972,0.01731,0.012241,0.058608,0.012241,...,0.012241,0.012241,0.146778,0.012241,0.108834,0.215249,0.024476,0.212398,0.073251,0.012241,0.092027,0.024476,0.168429,0.251551,0.024476,0.051867,0.467675,0.012241,0.012241,0.021198,0.012241,0.01731,0.029972,0.012241,0.137688
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.170732,0.333333,0.088235,0.278991,0.000000,0.154232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,0.268293,0.333333,0.205882,0.459243,0.000000,0.170137,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.414634,0.666667,0.352941,0.639495,0.000000,0.282919,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

MinMaxScaling 결과

- 평균이 0과 1 사이로 나오고, 표준편차도 다양하다.
- min/max 값은 0과 1로만 구성된다.

<br>

MinMaxScaling에서는 특히,  
다른 컬럼들에 비해 원-핫 인코딩을 통해 도출된 변수들은 스케일링을 했음에도 불구하고 변환값이 큰 거리를 가지게 되어 거리 기반 알고리즘을 적용할 때 너무 큰 역할을 하게 될 수도 있다는 것을 유의해야 한다.

In [332]:
# [참고] 학습과 변환을 한 번에 시킬 수 있다.
# ss.fit_transform(salary_df)  -> fit을 통해 학습된 평균/표준편차 값이 저장됨
# ss.transform() -> 예측 모델에서는 저장된 평균/표준편차를 가지고 실시간으로 예측하는 것이 가능

# 6. PCA

① PCA (Principal Component Analysis)

    차원 축소를 위한 기술  
    데이터의 주요 특성을 유지하면서 차원을 줄여 계산 효율성을 높인다.

② explained_variance_ratio_

    주로 주성분 분석(Principal Component Analysis, PCA)과 같은 차원 축소 기법에서 사용되는 속성  
    이 속성은 각 주성분이 원본 데이터의 분산(variance)을 얼마나 설명하는지에 대한 비율을 나타낸다.

In [334]:
from sklearn.decomposition import PCA

In [335]:
pca = PCA()

In [336]:
pca.fit(salary_df)

PCA()

In [337]:
salary_df

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
0,32.0,1,5.0,90000,0,132.00,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,28.0,2,3.0,65000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,45.0,3,15.0,150000,1,158.70,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36.0,1,7.0,60000,0,307.48,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,52.0,2,20.0,200000,0,307.48,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6669,49.0,3,20.0,200000,0,132.00,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6670,32.0,0,3.0,50000,0,135.30,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6671,30.0,1,4.0,55000,0,100.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6672,46.0,2,14.0,140000,0,100.00,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [338]:
pd.DataFrame(pca.transform(salary_df))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,-25294.401449,-35.927515,0.421192,1.180524,0.539384,-0.180507,0.681376,0.184226,1.002788,-0.353779,-0.100003,-0.181280,0.058951,-0.103092,-0.004072,-0.025935,-0.087458,-0.009350,-0.011387,0.025845,-0.027084,0.002889,0.131434,0.031248,-0.021312,...,0.000651,0.000269,-0.001381,-0.001743,3.827087e-13,-1.620587e-13,-1.446665e-13,3.287120e-13,-0.000134,0.000036,0.000176,-0.000281,0.000485,-0.000153,-0.000231,-0.000454,-0.000172,0.001163,0.000857,0.000820,-0.000196,0.002147,0.002123,-0.000175,7.725889e-14
1,-50294.407066,138.806380,-0.333187,0.224362,-0.958083,-0.241859,-0.241186,-0.383320,-0.143392,-0.010515,0.266753,0.168347,-0.250288,-0.028152,-0.013862,0.429985,0.133174,0.810935,0.153296,0.253917,0.751095,-0.122647,0.168979,-0.005033,0.150582,...,0.001170,0.002089,-0.000615,0.002337,7.592382e-13,4.316797e-13,-2.913167e-13,6.589747e-13,0.000006,-0.000326,-0.000181,0.000312,-0.000304,0.000211,0.000197,-0.000087,0.000091,-0.001037,-0.000251,-0.000776,0.000201,0.001553,0.001013,0.001235,1.500957e-13
2,34705.599441,-7.432077,8.552416,1.092177,-0.704550,-0.298237,0.195463,1.195063,-0.003915,-0.012052,-0.096195,0.414340,0.255642,-0.061686,-0.007109,0.017978,0.080600,0.017476,-0.002990,0.061125,0.173381,-0.017723,-0.292285,-0.339236,0.051027,...,-0.168270,-0.373371,-0.459276,-0.168128,-5.247650e-13,-1.652424e-14,1.988944e-13,-4.530381e-13,-0.000240,0.000371,0.000435,0.000088,-0.000029,0.000346,0.000302,0.000301,0.000116,0.000623,-0.000513,-0.000525,0.001282,0.001402,0.000892,-0.027603,-1.067883e-13
3,-55294.405818,138.644191,9.233644,1.028194,0.255635,-0.216322,-0.515753,-0.434514,-0.106573,0.012223,0.274566,0.192541,-0.501353,-0.078577,0.002643,-0.401954,0.500041,0.072504,0.006183,-0.237894,0.749731,0.008218,0.232856,-0.007099,0.089223,...,0.001033,0.001259,-0.000090,0.001921,8.351534e-13,3.694960e-13,-3.198747e-13,7.245585e-13,-0.000069,-0.000169,0.000090,0.000560,-0.000067,0.000011,-0.000130,-0.000118,0.000059,-0.000689,0.000207,-0.000112,0.000632,0.000912,-0.000132,0.001361,1.650837e-13
4,84705.595635,142.853086,10.357471,1.719041,0.880828,0.481965,-0.161033,-0.207667,-0.462547,-0.233319,-0.435806,-0.300143,-0.269897,-0.098484,-0.030532,0.116544,0.072956,0.018747,0.014937,0.034864,-0.236817,0.075935,-0.013262,-0.028159,-0.082385,...,-0.002686,-0.003554,0.000740,0.197234,2.500200e-01,-1.223090e-01,2.013751e-01,2.112714e-01,-0.044595,0.149696,-0.341390,-0.274571,0.060934,0.212014,0.256908,0.044788,-0.292259,-0.162288,-0.371234,0.197064,-0.248065,0.290574,0.118531,-0.076433,-2.675182e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6669,84705.600608,-32.627228,7.654235,0.031748,-0.428730,0.296013,-0.823992,-0.544413,0.631546,-0.487361,0.550570,-0.599993,0.289079,-0.123239,0.018158,0.030443,0.181065,-0.086645,0.713379,-0.110357,0.041772,-0.056600,-0.160873,0.024408,0.016186,...,-0.000688,-0.001536,0.000565,0.001906,-1.280381e-12,-7.789722e-13,4.872077e-13,-1.106101e-12,0.000080,-0.000324,-0.000152,0.000757,0.000047,-0.000439,-0.000482,-0.000603,0.000485,-0.001367,0.001040,0.001213,-0.001089,-0.001095,-0.001398,0.001801,-2.561897e-13
6670,-65294.401335,-33.835290,4.861691,1.979699,1.185291,0.089906,-0.103538,0.063824,-0.378510,-0.455633,0.276001,0.190400,-0.467971,0.928311,-0.011520,-0.445314,0.362428,-0.008605,-0.020795,-0.336310,0.007259,-0.120811,-0.093474,-0.078545,-0.002042,...,-0.000053,0.001924,-0.000017,-0.000680,9.881508e-13,9.985923e-14,-3.739237e-13,8.513278e-13,-0.000018,0.000054,0.000653,0.000265,-0.001080,0.000332,0.000165,-0.001030,-0.001251,0.000999,0.000386,-0.000112,-0.000358,-0.000158,-0.000364,-0.000072,1.984883e-13
6671,-60294.400430,-68.984421,3.005404,0.147405,-0.009182,-0.480902,-0.366391,-0.660713,-0.521658,0

In [339]:
pca = PCA(2)

In [340]:
pd.DataFrame(pca.fit_transform(salary_df), columns = ['PC1','PC2'])

,PC1,PC2
0,-25294.401449,-35.927515
1,-50294.407066,138.806380
2,34705.599441,-7.432077
3,-55294.405818,138.644191
4,84705.595635,142.853086
...,...,...
6669,84705.600608,-32.627228
6670,-65294.401335,-33.835290
6671,-60294.400430,-68.984421
6672,24705.601309,-66.436829


In [341]:
(pca.explained_variance_ratio_).sum()

0.9999999848414092

In [342]:
salary_df.corr()

,Age,Education Level,Years of Experience,Salary,Senior,CPI,Gender_Male,Country_Canada,Country_China,Country_UK,Country_USA,Race_Asian,Race_Australian,Race_Black,Race_Chinese,Race_Hispanic,Race_Korean,Race_Mixed,Race_Welsh,Race_White,Jobs_Accountant,Jobs_Administrative,Jobs_Advertising Coordinator,Jobs_Business Analyst,Jobs_Business Development,...,Jobs_Network Engineer,Jobs_Office Manager,Jobs_Operations,Jobs_PR,Jobs_Product Designer,Jobs_Product Manager,Jobs_Project Coordinator,Jobs_Project Engineer,Jobs_Project Manager,Jobs_Quality Assurance,Jobs_Receptionist,Jobs_Recruiter,Jobs_Research,Jobs_Sales,Jobs_Scientist,Jobs_Social Media,Jobs_Software,Jobs_Supply Chain,Jobs_Supply Chain Manager,Jobs_Technical Support Specialist,Jobs_Technical Writer,Jobs_Training Specialist,Jobs_UX,Jobs_Web Designer,Jobs_Web Developer
Age,1.000000,0.597131,0.938002,0.729071,0.333341,-0.027109,0.118189,0.005074,0.017223,0.011160,-0.025624,-0.018577,0.002827,0.012583,-0.004786,-0.025260,-0.001645,0.002743,0.006790,0.020116,-0.009004,0.008863,-0.005824,0.005692,0.021581,...,-0.005824,0.010296,0.038169,0.007072,-0.080869,-0.042186,0.017373,0.259979,0.012106,0.000624,-0.121778,0.004473,0.041298,-0.166507,0.017373,-0.044524,0.002858,0.026417,0.015133,0.006666,-0.010660,0.013423,0.013380,-0.007436,-0.068369
Education Level,0.597131,1.000000,0.612996,0.645249,0.274061,-0.016302,0.022686,0.001361,0.002490,0.022404,-0.016487,-0.004196,-0.019098,0.020257,-0.023535,-0.008992,0.010575,0.009533,0.015505,0.007718,-0.021197,-0.012234,-0.008650,-0.012513,0.005254,...,-0.008650,-0.008650,-0.086333,0.005254,-0.060630,-0.030196,-0.010351,0.184639,-0.045068,-0.008650,-0.171006,-0.017305,0.270508,-0.337510,0.038327,-0.036747,-0.034312,-0.008650,-0.008650,-0.014985,-0.008650,-0.002401,0.029911,-0.008650,-0.062120
Years of Experience,0.938002,0.612996,1.000000,0.811046,0.317443,-0.022393,0.099047,0.005579,0.013838,0.013330,-0.021354,-0.013785,-0.000897,0.013363,-0.009377,-0.023151,0.002661,0.003243,0.001373,0.019896,-0.016965,0.002652,-0.010307,-0.014528,0.022179,...,-0.010307,0.014057,0.021039,0.003905,-0.090000,-0.009336,0.011874,0.228036,0.011280,0.001875,-0.124326,-0.001326,0.100130,-0.202751,0.015936,-0.041389,0.000430,0.028270,0.020148,-0.001441,-0.012337,0.011267,0.005838,-0.010307,-0.102502
Salary,0.729071,0.645249,0.811046,1.000000,0.223751,-0.021693,0.127268,0.010821,0.008797,0.006277,-0.022481,-0.000178,-0.005398,0.015605,-0.020796,-0.020821,0.009268,0.004504,-0.011862,0.021137,-0.032352,-0.021404,-0.016293,-0.039523,0.012680,...,-0.012816,-0.011657,-0.041583,-0.005863,-0.111276,0.110118,-0.011149,0.213982,-0.006414,-0.003545,-0.158675,-0.025059,0.137077,-0.301000,0.003341,-0.058547,0.047428,0.003408,-0.002386,-0.018187,-0.016293,-0.010750,-0.003480,-0.016293,-0.147764
Senior,0.333341,0.274061,0.317443,0.223751,1.000000,-0.020193,0.039063,0.012860,0.014788,-0.001407,-0.020095,-0.014561,-0.016480,0.009976,-0.003624,-0.004145,-0.004055,-0.001556,-0.005208,0.028862,0.016297,-0.007075,-0.005002,0.048985,-0.005002,...,-0.005002,-0.005002,-0.032171,-0.005002,-0.025351,-0.078539,0.059927,0.543544,0.016642,0.029957,-0.037927,-0.010007,0.056142,-0.103466,0.042444,-0.021251,-0.059025,-0.005002,-0.005002,0.011521,-0.005002,0.017647,0.030574,-0.005002,-0.057370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jobs_Technical Writer,-0.010660,-0.008650,-0.012337,-0.016293,-0.005002,-0.005326,0.011089,-0.006081,-0.006124,-0.006107,-0.006179,0.021827,-0.003299,-0.003232,-0.003256,-0.002756,-0.003319,-0.002810,-0.002805,-0.007871,-0.000367,-0.000212,-0.000150,-0.000720,-0.000150,...,-0.000150,-0.000150,-0.001837,-0.000150,-0.001348,-0.002770,-0.000300,-0.002729,-0.000902,-0.000150,-0.001136,-0.000300,-0.002124,-0.003303,-0.000300,-0.000637,-0.008457,-0.000150,-0.000150,-0.000260,1.000000,-0.000212,-0.000367,-0.000150,-0.0